In [1]:
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import numpy as np

In [2]:
# 分類するクラス
classes = ['1', '2', '3', '4', '5', '6', '7', 
          '8', '9', '10', '11']

# それぞれ　egg_type_list の['Field', 'Undiscovered', 'Bug', 'Amorphous', 'Dragon', 'Fairy', 'Mineral', 
#                     'Flying', 'Grass', 'Human-Like', 'Monster', 'Water']　に対応


nb_classes = len(classes)

img_width, img_height = 150, 150

# トレーニング用とバリデーション用の画像格納先
train_data_dir = 'AllDataSet/train'
validation_data_dir = 'AllDataSet/validation'

# 今回はトレーニング用に664枚、バリデーション用に535枚の画像を用意した。
nb_train_samples = 664
nb_validation_samples = 534

batch_size = 16
nb_epoch = 10

In [3]:
def vgg_model_maker():
    """ VGG16のモデルをFC層以外使用。FC層のみ作成して結合して用意する """

    # VGG16のロード。FC層は不要なので include_top=False
    input_tensor = Input(shape=(img_width, img_height, 3))
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

    # FC層の作成
    top_model = Sequential()
    top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(nb_classes, activation='softmax'))

    # VGG16とFC層を結合してモデルを作成
    model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

    return model

In [4]:
def image_generator():
    """ ディレクトリ内の画像を読み込んでトレーニングデータとバリデーションデータの作成 """
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        zoom_range=0.2,
        horizontal_flip=True)

    validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    return (train_generator, validation_generator)

In [5]:
result_dir = 'results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [7]:
# モデル作成
vgg_model = vgg_model_maker()

# 最後のconv層の直前までの層をfreeze
for layer in vgg_model.layers[:15]:
    layer.trainable = False

# 多クラス分類を指定
vgg_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
                metrics=['accuracy'])

# 画像のジェネレータ生成
train_generator, validation_generator = image_generator()

# Fine-tuning
# history = vgg_model.fit_generator(
#     train_generator,
#     steps_per_epoch=nb_train_samples,
#     epochs=nb_epoch,
#     validation_data=validation_generator,
#     validation_steps=nb_validation_samples)

history = vgg_model.fit_generator(
    train_generator,
    steps_per_epoch=int(663/batch_size),
    epochs=nb_epoch,
    validation_data=validation_generator,
    validation_steps=int(534/batch_size)
)

vgg_model.save_weights(os.path.join(result_dir, 'finetuning.h5'))

Found 663 images belonging to 11 classes.
Found 534 images belonging to 11 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
41/41 [==============================] - ETA: 0s - loss: 2.5200 - accuracy: 0.1113

/Users/watanabeshingo/opt/anaconda3/lib/python3.8/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


41/41 [==============================] - 133s 3s/step - loss: 2.5200 - accuracy: 0.1113 - val_loss: 2.3630 - val_accuracy: 0.1364
Epoch 2/10
41/41 [==============================] - 126s 3s/step - loss: 2.3370 - accuracy: 0.1561 - val_loss: 2.3701 - val_accuracy: 0.1534
Epoch 3/10
41/41 [==============================] - 121s 3s/step - loss: 2.2912 - accuracy: 0.1901 - val_loss: 2.2652 - val_accuracy: 0.1913
Epoch 4/10
41/41 [==============================] - 137s 3s/step - loss: 2.2140 - accuracy: 0.2102 - val_loss: 2.2537 - val_accuracy: 0.2405
Epoch 5/10
41/41 [==============================] - 129s 3s/step - loss: 2.1233 - accuracy: 0.2365 - val_loss: 2.1238 - val_accuracy: 0.2670
Epoch 6/10
41/41 [==============================] - 136s 3s/step - loss: 1.9922 - accuracy: 0.3014 - val_loss: 2.2453 - val_accuracy: 0.1875
Epoch 7/10
41/41 [==============================] - 135s 3s/step - loss: 1.9454 - accuracy: 0.3091 - val_loss: 2.0187 - val_accuracy: 0.2955
Epoch 8/10
41/41 [======